## B92  Protocol
In this protocol, Alice sends qubits to Bob in either |0⟩ state or the |+⟩ state randomly. Unlike the BB84 protocol, the qubits sent are non-orthogonal. Alice keeps a record of the states she sends. Bob then measures the qubits in either the rectilinear basis or the diagonal basis. If the measurement result comes out to be the "down state" for any measurement, then Bob can tell which state must have been sent by Alice. He stores the indices of these measurements and sends them to Alice. The initial state corresponding to these indices would then comprise to be the shared key.  

In [1]:
from qiskit import * 
from qiskit.tools.visualization import circuit_drawer, plot_histogram
import numpy as np
import random

### Initial Qubit States
First, Alice makes a list recording the states she sends to Bob. Representing |0⟩ state with '0' and the |+⟩ state with '1'

In [2]:
#creating input states 
qr = QuantumRegister(1)
cr = ClassicalRegister(1) 


n = 8
inputStates = [0]*n
for i in range(n) : 
    inputStates[i] = random.randint(0,1)
print("List containing the information on qubit states sent by Alice:")
print(inputStates)



List containing the information on qubit states sent by Alice:
[0, 1, 1, 1, 1, 1, 0, 1]


### Main Body   
* Now, we implement the whole protocol as described in the beginning. In each iteration, Bob measures the qubit state and updates his key if he measures a "down" state. From the output of each iteration, one can verify if the protocol was implemented correctly.  
* The following code prints the state Alice sent, measurement basis used by Bob and the result he obtained. We can see Bob's key and the index list being updated in every iteration according to the measurement results

In [3]:
print("Input State:       0 => |0>   and   1 => |+>")
print("measurement basis: 0 => Rectilinear   and   1 => Diagonal basis")
print("-----------------------------------------------------------------")
measurement_bases = [0] * n
key_bob = []
key_index = []

for i in range(n) : 
    #generating random input states 0 ,+ 
    circuit = QuantumCircuit(qr,cr)  
    print("input state:",inputStates[i])
    if inputStates[i] == 1 : 
        circuit.h(qr[0])
    circuit.barrier()
    #print(circuit)
    
    
    #adding bob's measurement to the circuit
    measurement_bases[i] = random.randint(0,1)  #measurement choices for Bob
    print("measurement basis:",measurement_bases[i])
    if measurement_bases[i] == 1 : 
        circuit.h(qr[0])
    circuit.measure(qr,cr)
    
    
    
    
    #simulating the circuit
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend = backend, shots =1)
    results = job.result()
    answer = results.get_counts(circuit)
    output = int(list(answer.keys())[list(answer.values()).index(1)])
    print("Measurement Result:",output)   
    
    
    
    
    #detecting '1' measurement result and guessing input state
    if  output == 1 : 
        if measurement_bases[i] == 0 :
            key_index.append(i)
            key_bob.append(1)
        elif measurement_bases[i] == 1 :
            key_bob.append(0)
            key_index.append(i)
    
    print("Bob's key",key_bob)
    print("List of key indices that Bob shares with Alice",key_index)
    
    
    #Now bob sends key_index to alice
    
    
    #Alice makes a list of measurement corr to list of indices bob has sent
    key_alice = [0]*len(key_index)
    for j in range(len(key_index)) : 
        key_alice[j] = inputStates[key_index[j]]
    print("ALice's key", key_alice)
    print(circuit)
    print("-----------------------------------------------------------------")
        
    
    #Verifying the key
if key_alice == key_bob : 
    print("Key shared successfully")
print(key_alice)
        
    
    
    

Input State:       0 => |0>   and   1 => |+>
measurement basis: 0 => Rectilinear   and   1 => Diagonal basis
-----------------------------------------------------------------
input state: 0
measurement basis: 1
Measurement Result: 1
Bob's key [0]
List of key indices that Bob shares with Alice [0]
ALice's key [0]
       ░ ┌───┐┌─┐
q0_0: ─░─┤ H ├┤M├
       ░ └───┘└╥┘
c0_0: ═════════╩═
                 
-----------------------------------------------------------------
input state: 1
measurement basis: 1
Measurement Result: 0
Bob's key [0]
List of key indices that Bob shares with Alice [0]
ALice's key [0]
      ┌───┐ ░ ┌───┐┌─┐
q0_0: ┤ H ├─░─┤ H ├┤M├
      └───┘ ░ └───┘└╥┘
c0_0: ══════════════╩═
                      
-----------------------------------------------------------------
input state: 1
measurement basis: 0
Measurement Result: 1
Bob's key [0, 1]
List of key indices that Bob shares with Alice [0, 2]
ALice's key [0, 1]
      ┌───┐ ░ ┌─┐
q0_0: ┤ H ├─░─┤M├
      └───┘ ░ └╥┘
c0_0: ══